### import

In [1]:
import os, re, pickle, sys, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, Font, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
sys.path.append(r'../자산정리')
import functions

In [178]:
# 파일읽기
#############################################
wd=r"C:\Users\DATA\Desktop\압류추심요청서_2412"
fn_최신화결과 = "압류사건_최신화결과.xlsx"
company = "솔림"
cutoff = "241130" # 최근 전산조회
path_dataset = join(r"D:\3.자산\전산 dataset", company, cutoff, "pkl")
#############################################
당사명 = r'헬프|솔림' if company=="솔림" else "대성인베스트"
df_ori = pd.read_excel(join(wd, fn_최신화결과), dtype='str')
print(df_ori.최신화결과화면.value_counts())
print(df_ori.종국결과.value_counts())

    # 채무자, 보증인
path_debt = functions.키워드로파일명찾기(path_dataset, "채무자조회")
path_grt = functions.키워드로파일명찾기(path_dataset, "보증인새창.*새보증인상태") # 새보증인상태 : 시효, 불가는 추심 cf) 면책, 종료 # 새보증인상태는 종료면책불가인데 실제로는 조정인 경우만 체크됐음

debt_ori = pd.read_pickle(path_debt)
grt_ori = pd.read_pickle(path_grt)

최신화결과화면
Y    2883
Name: count, dtype: int64
종국결과
인용    2882
해제       1
Name: count, dtype: int64


In [183]:
# 재시작점
필요칼럼 = ["관할법원","사건번호","채권자목록","당사자","사건명","종국결과일","종국결과","최신화결과화면","사건키","일반내용","진행내용","채무자키"]

debt = debt_ori[["채무자키","성명","주민번호인","채무상태","담당자","변경전성명인"]].rename(columns={"채무상태":"상태","주민번호인":"주민번호","성명":"당사자"})
grt = grt_ori[["채무자키","보증인키","보증인성명인","주민번호인","보증인상태","담당자"]].rename(columns={"보증인성명인":"당사자","보증인상태":"상태","주민번호인":"주민번호"})
df = df_ori[필요칼럼].copy()

# 채무자와 보증인 합치기, 주민번호식별처리, 상태값
채무자 = pd.concat([debt,grt], axis=0) # 행으로 연결
    # 채무자키와 주민번호를 조합하여 채무자키별로 고유한 사람만 남기기
채무자 = 채무자.drop_duplicates(["채무자키","주민번호"], keep='first')
채무자["주민번호"] = 채무자["주민번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))

# 전처리
    # 중복사건 제거
print(len(df))
df = df.drop_duplicates(subset=["관할법원","사건번호","당사자"],keep='first')
print(len(df))
    # 최신화 실패건, 종국결과 인용(~'해제'포함) 제거
df = df.query('최신화결과화면=="Y" and 종국결과=="인용"')
    # 사건명에서 "[전자] " 제거
df["사건명"] = df["사건명"].str.replace(r"\[전자\] ","", regex=True)
    # 당사여부 표시
df["당사건여부"] = df["채권자목록"].str.contains(당사명)

# 병합 : 주민번호, 상태, 담당자
print("병합전 df:",len(df))
rst = df.merge(채무자[["채무자키","보증인키","당사자","주민번호","상태","담당자"]], on=["채무자키","당사자"], how='left') # 주민번호인,상태,담당자 생성 / 변경전성명인은 미병함
print("병합후 df:",len(rst))

# 병합실패건 변경전성명인에서 한번 더 확인해보았으나 매칭 되는 거 없네. 출력용 파일에서 직접 수정(최신화결과 파일에서 수정해봐야 여기선 ==를 사용하기 때문에 또다시 매칭 안 될 거다.)
print('병합(매칭)실패건 :', len(rst.loc[rst.주민번호.isna()]))

2883
2883
병합전 df: 2882
병합후 df: 2882
병합(매칭)실패건 : 17


In [184]:
# 제3채무자로 행분열

    # 정규식 패턴 정의
pattern = r"제3채무자\d{1,2}\s?(.*?)에게"

    # 문자열 추출 및 새로운 데이터프레임 생성
rst["제3채무자_추출"] = rst["진행내용"].apply(
    lambda x: list(set(re.findall(pattern, x))) if re.findall(pattern, x) else ["진행내용 조회실패"])

    # 기존 행을 반복하면서 제3채무자 정보를 새 행으로 분리
expanded_rows = []
for idx, row in tqdm(rst.iterrows(), total=len(rst)):
    extracted_values = row["제3채무자_추출"]
    
    for i, value in enumerate(extracted_values):
        new_row = row.copy()  # 기존 행 복사
        new_row["제3채무자"] = value  # 새로운 "제3채무자" 열에 값 추가
        expanded_rows.append(new_row)

    # 새로운 데이터프레임 생성
expanded_df = pd.DataFrame(expanded_rows).drop(columns=["제3채무자_추출"])
print('확장 후 :',len(expanded_df))

100%|██████████| 2882/2882 [00:01<00:00, 1855.24it/s]


확장 후 : 8845


In [185]:
# 후처리 후 출력
    # 제3채무자별로 정렬
expanded_df = expanded_df.sort_values(by=["제3채무자","관할법원","사건번호","채무자키","당사자"])
    # 당사건 분리
당사건 = expanded_df[expanded_df.당사건여부].copy()
타사건 = expanded_df[~expanded_df.당사건여부].copy()

# 순번열 추가
당사건["순번"] = range(1,len(당사건)+1)
타사건["순번"] = range(1,len(타사건)+1)

# 출력
    # 출력칼럼
cols_output = ["순번","채무자키","보증인키","상태","담당자","채권자목록", "당사자", "주민번호", "관할법원", "사건번호", "종국결과일", "제3채무자"]
    # 출력
print(f'당사 {len(당사건)}건, 타사 {len(타사건)}건, 엑셀파일 출력')
functions.save_df_to_excel_underline(당사건[cols_output], join(wd, "제3채무자별_압류사건_당사_1209.xlsx"), key_columns_no=0)
functions.save_df_to_excel_underline(타사건[cols_output], join(wd, "제3채무자별_압류사건_타사_1209.xlsx"), key_columns_no=0)

당사 3981건, 타사 4864건, 엑셀파일 출력


### 결정문 파일 경로 확보

In [31]:
# 파일읽기
#############################################
wd=r"C:\Users\DATA\Desktop\압류추심요청서_2412"
fn = "제3채무자별_압류사건_당사_1209.xlsx"
sheet_name = '발송_기타'
#############################################
df_ori = pd.read_excel(join(wd, fn), sheet_name=sheet_name, dtype='str')

#### 재시작

In [32]:
df1 = df_ori.copy()
df1 = df1.drop_duplicates(["채무자키","사건번호"])

In [33]:
# 파일경로 읽기
#####################################
src_dir = r"C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행"
#####################################
event = re.compile('\d{2,4}타채\d{0,10}')

file_list = []
for r, d, files in os.walk(src_dir) : 
    for f in files : 
        사건번호검색결과 = event.search(f)
        if f.startswith('2') and 사건번호검색결과 : 
            file_list.append([f[:8], 사건번호검색결과[0], join(r,f)]) # 채무자키, 사건번호, 파일명

파일경로 = pd.DataFrame(file_list, columns=["채무자키","사건번호","파일경로"])

In [34]:
# 파일경로 중복값제거
파일경로 = 파일경로.drop_duplicates(subset=["채무자키","사건번호"])

In [35]:
print("병합전", len(df1))
경로병합 = df1[["순서","채무자키","당사자","사건번호","제3채무자"]].merge(파일경로, on=["채무자키","사건번호"], how='left')
print("병합후", len(경로병합))

병합전 909
병합후 909


In [36]:
경로병합[경로병합.파일경로.isna()]

,순서,채무자키,당사자,사건번호,제3채무자,파일경로


#### case1) 파일 없는 경우 실행

In [16]:
dst_dir = join(src_dir, "부분파일명")
if not os.path.exists(dst_dir) :
    os.mkdir(dst_dir)
print(len(경로병합[경로병합.파일경로.isna()]))

for i, v in 경로병합[경로병합.파일경로.isna()].iterrows() : 
    검색폴더 = join(src_dir, v.채무자키)
    try :
        files = os.listdir(검색폴더)
    except :
        print(v.채무자키, v.당사자, v.사건번호)
        continue
    for f in files : 
        부분일치검색결과 = re.search(v.사건번호[:6], f) # 연도까지만 
        if re.search(v.당사자, f) and 부분일치검색결과 :
            fn_new = v.채무자키 + " " + v.당사자 + " " + v.사건번호 + ".pdf"
            dst_dir_2 = join(dst_dir, v.채무자키) 
            
            if not os.path.exists(dst_dir_2) :
                os.mkdir(dst_dir_2)
            shutil.copy2(join(검색폴더,f), join(dst_dir_2,fn_new))

43
20411378 김영철 2019타채32464


#### case2) 파일 없는 거 실행 후에도 없는 건 수작업으로

In [38]:
경로병합[경로병합.파일경로.isna()].to_excel(join(src_dir,"사건번호불명.xlsx"), index=False)

#### case3) 파일경로 빈값 없으면

In [37]:
# df1에 경로병합
print(len(df_ori))
df_ori_경로병합 = df_ori.merge(경로병합[["채무자키","사건번호","파일경로"]], on=["채무자키","사건번호"], how='left')
print(len(df_ori_경로병합))

2143
2143


In [38]:
df_ori_경로병합.head()

,순서,채무자키,보증인키,상태,담당자,채권자목록,당사자,주민번호,관할법원,사건번호,종국결과일,제3채무자,공백제거,빈도수,파일경로
0,0001,20413663,NaN,정상,김효중,1.주식회사헬프자산관리대부대표이사정호석\n채무자\n\n2020.07.09\n1.대한...,가순경,570906-2******,대전지방법원 천안지원,2020타채105653,2020-05-26,대한민국 - 우정사업본부,대한민국-우정사업본부,375,C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행\2041...
1,0002,20417082,NaN,정상,박민재,1.주식회사헬프자산관리대부대표자사내이사정호석\n채무자\n\n2019.11.13\n2...,감문익,660205-2******,울산지방법원,2019타채112982,2019-11-08,대한민국 - 우정사업본부,대한민국-우정사업본부,375,C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행\2041...
2,0003,20411375,200186026,정상,박민재,1.주식회사헬프자산관리대부대표이사정호석\n채무자\n\n2018.11.12\n2.대한...,강건철,581112-1******,평택지원,2018타채45776,2018-11-08,대한민국 - 우정사업본부,대한민국-우정사업본부,375,C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행\2041...
3,0004,20411324,NaN,정상,도현관,1.주식회사헬프자산관리대부대표이사정호석\n채무자\n\n2019.06.21\n2.대한...,강미자,550823-2******,부산지방법원 동부지원,2019타채105323,2019-06-17,대한민국 - 우정사업본부,대한민국-우정사업본부,375,C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행\2041...
4,0005,20423252,NaN,정상,조준일,1.주식회사솔림헬프자산관리대부대표이사정호석\n채무자\n\n2021.11.16\n2....,강성대,651130-1******,울산지방법원,2021타채124512,2021-11-11,대한민국 - 우정사업본부,대한민국-우정사업본부,375,C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행\2042...


In [39]:
df_ori_경로병합.순서

0       0001
1       0002
2       0003
3       0004
4       0005
        ... 
2138    2139
2139    2140
2140    2141
2141    2142
2142    2143
Name: 순서, Length: 2143, dtype: object

In [40]:
# df1 순서에 맞추어 파일복사
dst_base = r"C:\Users\DATA\Desktop\압류추심요청서_2412\결정문\241216_기타금융"
for i, v in tqdm(df_ori_경로병합.iterrows(), total=len(df_ori_경로병합)) :
    src = v.파일경로
    dst_dir = dst_base # join(dst_base, v.제3채무자)
    if not os.path.exists(dst_dir) : 
        os.mkdir(dst_dir)
    
    dst_fn = v.순서 + " " + v.당사자 + " " + v.사건번호 + ".pdf" # 순서 때문에 중복값은 없음
    # 파일 복사
    shutil.copy(src, join(dst_dir,dst_fn))

100%|██████████| 2143/2143 [00:04<00:00, 474.38it/s]


### 결정문 인쇄

In [ ]:
import os
import win32print
import win32api
import time

def print_pdfs_in_folder(folder_path):
    # 폴더에서 PDF 파일만 가져오기
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
    
    # 파일명 기준으로 정렬
    pdf_files.sort()

    # 프린터 정보 가져오기
    printer_name = win32print.GetDefaultPrinter()
    
    print(f"사용 중인 기본 프린터: {printer_name}")

    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)

        # PDF 파일 인쇄
        try:
            result = win32api.ShellExecute(0, "print", pdf_path, None, folder_path, 0)
            if result > 32:  # 성공적인 요청 코드 확인
                print(f"{pdf_file} 인쇄 요청 성공")
                time.sleep(5)  # 인쇄 요청 간 대기 시간 추가
            else:
                print(f"{pdf_file} 인쇄 요청 실패: 코드 {result}")
        except Exception as e:
            print(f"{pdf_file} 인쇄 실패: {e}")



folder_path = r"C:\Users\DATA\Desktop\압류추심요청서_2412\결정문\241211_1금융\주식회사 우리은행"

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    print_pdfs_in_folder(folder_path)
else:
    print("유효한 폴더 경로를 입력해주세요.")

사용 중인 기본 프린터: iR-ADV 4725
인쇄 요청 중: 0821 서수곤 2021타채103288.pdf
0821 서수곤 2021타채103288.pdf 인쇄 요청 성공
인쇄 요청 중: 0822 서진호 2021타채108476.pdf
0822 서진호 2021타채108476.pdf 인쇄 요청 성공
인쇄 요청 중: 0823 안준섭 2019타채105710.pdf
0823 안준섭 2019타채105710.pdf 인쇄 요청 성공
인쇄 요청 중: 0824 김영남 2019타채113043.pdf
0824 김영남 2019타채113043.pdf 인쇄 요청 성공
인쇄 요청 중: 0825 신영섭 2019타채116710.pdf
0825 신영섭 2019타채116710.pdf 인쇄 요청 성공
인쇄 요청 중: 0826 피혁진 2020타채123826.pdf
0826 피혁진 2020타채123826.pdf 인쇄 요청 성공
인쇄 요청 중: 0827 박영태 2021타채103946.pdf
0827 박영태 2021타채103946.pdf 인쇄 요청 성공
인쇄 요청 중: 0828 변혜정 2021타채127097.pdf
0828 변혜정 2021타채127097.pdf 인쇄 요청 성공
인쇄 요청 중: 0829 최수아 2020타채50819.pdf
0829 최수아 2020타채50819.pdf 인쇄 요청 성공
인쇄 요청 중: 0830 강지선 2020타채63087.pdf
0830 강지선 2020타채63087.pdf 인쇄 요청 성공
인쇄 요청 중: 0831 정원철 2021타채69226.pdf
0831 정원철 2021타채69226.pdf 인쇄 요청 성공
인쇄 요청 중: 0832 최병호 2021타채74718.pdf
0832 최병호 2021타채74718.pdf 인쇄 요청 성공
인쇄 요청 중: 0833 유성준 2021타채74740.pdf
0833 유성준 2021타채74740.pdf 인쇄 요청 성공
인쇄 요청 중: 0834 황요하 2020타채103486.pdf
0834 황요하 2020타채103486.pdf 인쇄 요청 성공
인쇄 요